## 1. UserId (게시글별 ID) 크롤러

In [4]:
# library import
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.chrome.options import Options
import time
import requests
import pickle
from bs4 import BeautifulSoup

# selenium 속도 향상 위해 불필요한 옵션을 사용하지 않게하는 코드
options = Options()
prefs = {'profile.default_content_setting_values': {'cookies' : 2, 'images': 2, 
                                                    'plugins' : 2, 'popups': 2, 'geolocation': 2,
                                                    'notifications' : 2, 'auto_select_certificate': 2,
                                                    'fullscreen' : 2,
                                                    'mouselock' : 2, 'mixed_script': 2, 
                                                    'media_stream' : 2, 'media_stream_mic' : 2,
                                                    'media_stream_camera': 2, 'protocol_handlers' : 2,
                                                    'ppapi_broker' : 2, 'automatic_downloads': 2, 'midi_sysex' : 2,
                                                    'push_messaging' : 2, 'ssl_cert_decisions': 2, 'metro_switch_to_desktop' : 2,
                                                    'protected_media_identifier': 2, 'app_banner': 2, 'site_engagement' : 2,
                                                    'durable_storage' : 2}}

options.add_experimental_option('prefs', prefs) 
options.add_argument("start-maximized") 
options.add_argument("disable-infobars") 
options.add_argument("--disable-extensions")

## 카테고리 별 게시글 리스트 페이지에서 유저별 id 파싱 (*전체 18개 카테고리)
## 각 페이지별 무한 스크롤 javaScript 제어를 위한 셀레니움 기능 사용
def get_user_list(base_url):
#     크롬드라이버 설정
    chromedriver = 'C:/selenium/chromedriver.exe' 
    driver = webdriver.Chrome(chromedriver)
    driver.get(base_url) 
    
    SCROLL_PAUSE_TIME = 15 # 무한스크롤 멈춤 현상 방지
    last_height = driver.execute_script("return document.body.scrollHeight")
    i = 0
    while True:
        # Scroll down to bottom                                                      
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)                                                
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")  
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height            
        new_height = driver.execute_script("return document.body.scrollHeight")
        i+=1

        if i == 700 :                                            
            break
        last_height = new_height
    
    print(i,"회에서 멈춤")
    
    ## 각 페이별 700번 스크롤 후, BeautifulSoup으로 "전체 게시글 url parsing"
    html = driver.page_source
    soup = BeautifulSoup(html,'html.parser')
    
    # 게시글별 user_id parsing
    a_tags = soup.select('#wrapArticle > div.wrap_article_list.\#keyword_related_contents > ul > li > a.link_post')
    
    save_href = []
    for a_tag in a_tags :
        save_href.append(a_tag['href'])
    
    driver.close()
    
    return save_href # 개별 카테고리별 전체 게시글 url 반환

######################### 
category_list = ['지구한바퀴_세계여행?q=g','시사·이슈?q=g','IT_트렌드?q=g',
                '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g',
                 '직장인_현실_조언?q=g','스타트업_경험담?q=g','육아_이야기?q=g',
                 '요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
                 '문화·예술?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g',
                '우리집_반려동물?q=g','사랑·이별?q=g','감성_에세이?q=g']


# 전체 18개 카테고리별로 돌며 게시글 url 파싱하여 pickle로 저장
for category in category_list:
    each_user_id = []
    each_user_id = get_user_list("https://brunch.co.kr/keyword/"+category)
    with open(category[:-4]+'_userId.txt','wb') as f:
        pickle.dump(each_user_id,f)

## 2. 각 게시글별 정보 크롤러

In [ ]:
import pandas as pd
import pickle
import requests
import json
import re
from tqdm import tqdm
from bs4 import BeautifulSoup

## url pickle load
pickles = ['지구한바퀴_세계여행?q=g','시사·이슈?q=g','IT_트렌드?q=g',
                '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g',
                 '직장인_현실_조언?q=g','스타트업_경험담?q=g','육아_이야기?q=g',
                 '요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
                 '문화·예술?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g',
                '우리집_반려동물?q=g','사랑·이별?q=g','감성_에세이?q=g']


writer_list = []
for file in pickles:
    with open(file[:-4]+"_userId.txt","rb") as fr:
        writers = pickle.load(fr)
    writer_list.append(writers)  ## [[카테고리1 게시글 url...],[카테고리2 게시글 url], ....[카테고리24 게시글 url]]

## 게시글 속 정보 수집
def def_craw(writer):
    
    json_data = {}
    data = []
    res_text = []
    tag_keyword=[]
    
    tag_title,tag_nickname,tag_publish_date,tag_url,tag_url_plink = None,None,None,None,None
    tag_share,tag_like = str,str
    for idx,url in enumerate(writer):
        if idx % 500 == 0 : print("전체",len(writer),"중에",idx)
        if res_text == []: # 첫 시작에러 방지
            pass
        else :
            # to json
            json_data['title'] = tag_title  
            json_data['nickname'] = tag_nickname
            json_data['publish_date'] = tag_publish_date
            json_data['keyword'] = tmp_keyword   
            json_data['like'] = tag_like # like 없는 경우 ''
            json_data['share'] = tag_share # share 없는 경우 None            
            json_data['comment'] = tag_comment # comment 없는 경우 ''
            json_data['url'] = tag_url
            json_data['url_plink'] = tag_url_plink 
            json_data['text'] = res_text

        data.append(json_data)
        
        json_data = {} # 누적방지 초기화
        tmp_keyword = [] # 누적방지 초기화
        res_text = [] # 누적방지 초기화 
        
        # beautifulsoup
        html = requests.get('https://brunch.co.kr{text_url}'.format(text_url=url))
        soup = BeautifulSoup(html.text, 'html.parser')
        
        if soup.find('title').text == "brunch":
            pass
        else:
            tag_title = soup.find('title').text # 게시글 title
            tag_url = soup.find("meta",property='og:url')['content'] # 게시글 본주소
            tag_nickname = soup.find("meta",{'name':'article:media_name'})['content'] # 작가 nickname
            tag_url_plink = soup.find("meta",property='dg:plink')['content'] # 암호주소? # 모바일?
            tag_publish_date = soup.find("meta",property='article:published_time')['content'] # 발행일
            tag_keyword = soup.find_all('a',href=re.compile('/keyword')) # 게시글 키워드
            tag_like = soup.find('span',{'class':'f_l text_like_count text_default text_with_img_ico ico_likeit_like #like'}) #좋아요 수
            tag_share = soup.find('span',{'class':'f_l text_share_count text_default text_with_img_ico'}) # 공유 수
            tag_comment = soup.find('span',{'class':'f_l text_comment_count text_default text_with_img_ico'}) # 댓글 수
            text_h4 = soup.find_all(class_='wrap_item item_type_text')
            
            for text in text_h4:
                res_text.append(text.text)
    
            if tag_like == None:
                tag_like = "0"
            else:
                tag_like = tag_like.text # 좋아요 수

            if tag_share == None:
                tag_share == "0"
            else:
                tag_share = tag_share.text # 공유 수

            if tag_comment == None:
                tag_comment =="0"
            else:
                tag_comment = tag_comment.text

            for keyword in tag_keyword:
                tmp_keyword.append(keyword.text)
                
    return data ## 수집한 정보를 담은 dictionary로 반환

categories = ['지구한바퀴_세계여행','시사_이슈','IT_트렌드',
                '취항저격_영화리뷰','오늘은_이런책','뮤직_인사이드',
                 '직장인_현실조언','스타트업_경험담','육아_이야기',
                 '요리_레시피','건강_운동','멘탈관리_심리탐구',
                 '문화_예술','인문학_철학','쉽게_읽는_역사',
                '우리집_반려동물','사랑_이별','감성_에세이']


## 카테고리 -> 게시글의 순서로 2차 크롤링 진행
## 카테고리별로 정보를 담은 json 형식으로 저장(총 24개 json file)
from collections import OrderedDict
for idx,writer in enumerate(writer_list):
    to_json = None
    data = def_craw(writer) # 2단계 크롤링 실행
    
    del data[0]
    del data[0]
    
    to_json = OrderedDict()
    to_json['name'] = categories[idx] # category name
    to_json['version'] = "2020-06-01"
    to_json['data'] = data
    
    with open(categories[idx]+".json","w") as make_file:
        json.dump(to_json,make_file)

## 3. Keyword 크롤러

In [ ]:
import pandas as pd
import pickle
import requests
import json

from bs4 import BeautifulSoup

category = ['지구한바퀴_세계여행?q=g','시사·이슈?q=g','IT_트렌드?q=g',
                '취향저격_영화_리뷰?q=g','오늘은_이런_책?q=g','뮤직_인사이드?q=g',
                 '직장인_현실_조언?q=g','스타트업_경험담?q=g','육아_이야기?q=g',
                 '요리·레시피?q=g','건강·운동?q=g','멘탈_관리_심리_탐구?q=g',
                 '문화·예술?q=g','인문학·철학?q=g','쉽게_읽는_역사?q=g',
                '우리집_반려동물?q=g','사랑·이별?q=g','감성_에세이?q=g']

name_list = ['지구한바퀴_세계여행','시사_이슈','IT_트렌드','취향저격_영화리뷰',
             '오늘은_이런책','뮤직_인사이드','직장인_현실조언','스타트업_경험담',
             '육아_이야기','요리_레시피','건강_운동','멘탈관리_심리탐구','문화_예술',
             '인문학_철학','쉽게_읽는_역사','우리집_반려동물','사랑_이별','감성_에세이']

keyword_dict = {}

## 현재 브런치에서 각 카테고리별로 제공하는 keyword 크롤링하여 dict type으로 저장
for idx,cate in enumerate(category):
    name_list[idx]
    html = requests.get('https://brunch.co.kr/keyword/{category_name}'.format(category_name=cate))
    soup = BeautifulSoup(html.text, 'html.parser')
    
    target = soup.find_all(class_ = 'keyword_elem')
    
    keywords = []
    for i in target :
        keywords.append(i.text)
    
    keyword_dict[name_list[idx]] = keywords
    
# 스타트업_경험담 카테고리의 경우에 keyword 정보가 없기 때문에 직접 지정.
del(keyword_dict['스타트업_경험담'])
keyword_dict['스타트업_경험담'] = ['스타트업','창업','이직','마케팅','조직','캠퍼스','패스트파이브']

with open ('keyword_dict.txt','wb') as f:
    pickle.dump(keyword_dict,f)